In [ ]:
1from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import cv2
import tqdm
import heapq
import datetime
import glob
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [ ]:
TRAIN_PATH = '/content/drive/My Drive/UCF-CV_Project/Training_set/Training'
TEST_PATH = '/content/drive/My Drive/UCF-CV_Project/Testing_set'

In [ ]:
import os
import cv2
import tensorflow as tf
import tqdm
import numpy as np


def data_generator():
    for index, category in enumerate(os.listdir(TRAIN_PATH)):
        os.mkdir(category)
        category_path = os.path.join(TRAIN_PATH, category)
        for video in os.listdir(category_path):
            video_path = os.path.join(category_path, video)
            cap = cv2.VideoCapture(video_path)
            num_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            num = max(1, num_of_frames // 50)
            frame_idx = 0
            taken = 0
            while(1):
                ret, frame = cap.read()
                if(ret == False):
                  break
                if frame_idx % num == 0:
                    frame = frame[:, :, ::-1]
                    img = tf.image.resize(frame, (299, 299))
                    img = tf.keras.applications.inception_v3.preprocess_input(img)
                    taken += 1
                    yield img, video_path
                if(taken == 50):
                  break
                frame_idx += 1


dataset = tf.data.Dataset.from_generator(data_generator,
             output_types=(tf.float32, tf.string),
             output_shapes=((299, 299, 3), ()))

dataset = dataset.batch(16).prefetch(tf.data.experimental.AUTOTUNE)


inception_v3 = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')
inception_v3_output = inception_v3.output
pooling_output = tf.keras.layers.GlobalAveragePooling2D()(inception_v3_output)
feature_extraction_model = tf.keras.Model(inception_v3.input, pooling_output)

current_path = None

all_features = []

for img, batch_paths in tqdm.tqdm(dataset):
    batch_features = feature_extraction_model(img)
    batch_features = tf.reshape(batch_features, (batch_features.shape[0], -1))
    
    for features, path in zip(batch_features.numpy(), batch_paths.numpy()):
        if path != current_path and current_path is not None:

            video = os.path.split(current_path.decode())[-1]
            category = os.path.split(os.path.split(current_path.decode())[-2])[-1]
            print(os.path.join(category, video))
            np.save(os.path.join(category, video.replace('.mpg', '.npy') ), all_features)
            print(len(all_features))
            all_features = []

        current_path = path
        all_features.append(features)

In [ ]:
categories_list = ['/content/Basketball', 
          '/content/Diving',
          '/content/Jumping',
          '/content/Tennis',
          '/content/Walking']

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

df = pd.DataFrame(columns=['category', 'id', 'path'])

for index, category_path in enumerate(categories_list):
    for video in os.listdir(category_path):
        video_path = os.path.join(category_path, video)
        category = os.path.dirname(video_path).split(os.sep)[-1]    
        df = df.append({'category':category,  'id': index, 'path': video_path}, ignore_index=True)

trainfile = open('train.txt', 'w')
testfile = open('test.txt', 'w')
categories = df.category.unique()

LABELS = categories
encoder = LabelBinarizer()
encoder.fit(categories)
print(encoder)
print('Classes: {}'.format(len(encoder.classes_)))

for idx in range(5):    
    class_subset = df[df['category'] == categories[idx]]
    train, test = train_test_split(class_subset, test_size=0.2)    

    for index, row in train.iterrows():
        print(index, row)
        trainfile.write(row['path'] + ' ' + str(row['id']) + '\n')
        
    for index, row in test.iterrows():
        testfile.write(row['path'] + '\n')        
trainfile.close()
testfile.close()

In [ ]:
test_file = os.path.join('content', 'test.txt')
train_file = os.path.join('content', 'train.txt')

with open('/content/test.txt') as f:
    test_list = [row.strip() for row in list(f)]

with open('/content/train.txt') as f:
    train_list = [row.strip() for row in list(f)]
    train_list = [row.split(' ')[0] for row in train_list]

def make_generator(file_list):
    def generator():
        np.random.shuffle(file_list)
        for path in file_list:
            full_path = path
            #print(full_path [-4:])
            if(full_path [-4:] != '.npy'):full_path += '.npy'
            #print('hi', full_path)

            label = os.path.basename(os.path.dirname(path))
            features = np.load(full_path)

            padded_sequence = np.zeros((50, 2048))
            padded_sequence[0:len(features)] = np.array(features)

            transformed_label = encoder.transform([label])
            yield padded_sequence, transformed_label[0]
    return generator

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Masking(mask_value=0.),
    tf.keras.layers.LSTM(512, dropout=0.5, recurrent_dropout=0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(5, activation='softmax')])

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy', 'top_k_categorical_accuracy'])

In [ ]:
train_dataset = tf.data.Dataset.from_generator(make_generator(train_list),
                 output_types=(tf.float32, tf.int16),
                 output_shapes=((50, 2048), (len(categories))))
train_dataset = train_dataset.batch(16).prefetch(tf.data.experimental.AUTOTUNE)


valid_dataset = tf.data.Dataset.from_generator(make_generator(test_list),
                 output_types=(tf.float32, tf.int16),
                 output_shapes=((50, 2048), (len(categories))))
valid_dataset = valid_dataset.batch(16).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
import heapq
import datetime
import glob
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir="logs\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = TensorBoard(log_dir=log_dir, update_freq=1000, profile_batch=0)
checkpoint = ModelCheckpoint("model.h5", verbose=1, save_best_only=True)
earlystopping = EarlyStopping(patience=10, verbose=1)
model.fit(train_dataset, epochs=100, callbacks=[tensorboard_callback, checkpoint, earlystopping], validation_data=valid_dataset)

In [ ]:
for index, video in enumerate(os.listdir(TEST_PATH)):
        video_path = os.path.join(TEST_PATH, video)
        print(index, video_path)

In [ ]:
def TEST_data_generator():
    #os.mkdir('TEST')
    for index, video in enumerate(os.listdir(TEST_PATH)):
            video_path = os.path.join(TEST_PATH, video)
            cap = cv2.VideoCapture(video_path)
            num_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            num = max(1, num_of_frames // 50)
            frame_idx = 0
            taken = 0
            while(1):
                ret, frame = cap.read()
                if(ret == False):
                  break
                if frame_idx % num == 0:
                    frame = frame[:, :, ::-1]
                    img = tf.image.resize(frame, (299, 299))
                    img = tf.keras.applications.inception_v3.preprocess_input(img)
                    taken += 1
                    yield img, video_path
                if(taken == 50):
                  break
                frame_idx += 1


TEST_dataset = tf.data.Dataset.from_generator(TEST_data_generator,
             output_types=(tf.float32, tf.string),
             output_shapes=((299, 299, 3), ()))

TEST_dataset = TEST_dataset.batch(16).prefetch(tf.data.experimental.AUTOTUNE)


inception_v3 = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')
inception_v3_output = inception_v3.output
pooling_output = tf.keras.layers.GlobalAveragePooling2D()(inception_v3_output)
feature_extraction_model = tf.keras.Model(inception_v3.input, pooling_output)

current_path = None

all_features = []

for img, batch_paths in TEST_dataset:
    batch_features = feature_extraction_model(img)
    batch_features = tf.reshape(batch_features, (batch_features.shape[0], -1))
    
    for features, path in zip(batch_features.numpy(), batch_paths.numpy()):
        if path != current_path and current_path is not None:
            video = os.path.split(current_path.decode())[-1]
            print(current_path)
            np.save(os.path.join('TEST', video.replace('.mpg', '.npy') ), all_features)
            all_features = []

        current_path = path
        all_features.append(features)

video = os.path.split(current_path.decode())[-1]
print(current_path)
np.save(os.path.join('TEST', video.replace('.mpg', '.npy') ), all_features)


In [ ]:
def load_features(path):
    features = np.load(path)
    padded_sequence = np.zeros((50, 2048))
    padded_sequence[0:len(features)] = np.array(features)
    return np.array([padded_sequence])

categories_mapping = {1: 0,
                      3: 3,
                      2: 1,
                      4: 4,
                      0: 2
                      }

videos = []
labels = []
i = 0
for file in os.listdir('/content/TEST'):
    i += 1
    print(file)
    file_path = os.path.join('/content/TEST', file)
    features = load_features(file_path)
    prediction = model.predict(features)
    #print(np.argmax(prediction), categories_mapping[np.argmax(prediction)])
    videos.append(file.replace('npy', 'mpg'))
    labels.append(categories_mapping[np.argmax(prediction)])



print(i)
with open('submit.csv','w') as file:
    file.write('Video,')
    file.write('Label')
    file.write('\n')
    for i in range(len(videos)):
        file.write(videos[i] + ',')
        file.write(str(labels[i]))
        file.write('\n')